# Explorando convolução no Keras usando o Theano

O objetivo deste notebook é de explorar a convolução que é implementada pelo Theano através do Keras.

A convolução é a operação essencial das redes convolucionais. É uma operação demorada e a sua implementação na GPU acelera bastante o seu processamento. A implementação é feita no Theano utilizando GPU. Se a GPU não for encontrada, a implementação roda na CPU.

[A guide to convolution arithmetic for deep
learning by Vincent Dumoulin and Francesco Visin, 2016](https://arxiv.org/pdf/1603.07285.pdf)


In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.utils.np_utils import convert_kernel



Using Theano backend.


# Uma rede com uma única camada convolucional

O objetivo deste experimento é explorar numericamente a convolução implementada na rede
convolucional do Keras/Theano. Para isso iremos criar uma rede convolucional com apenas
uma camada de convolução. A sua predição será o resultado da rede, que neste caso será apenas uma convolução.

Uma convolução pode ser visto como uma média móvel ponderada sobre a imagem. A equação matemática da convolução é dada por:

$$ \boldsymbol{Y} = \boldsymbol{X} \ast \boldsymbol{W} $$

onde $x$ é a imagem de entrada, $y$ é o resultado da convolução e $w$ é chamado de peso,
núcleo, máscara, entre outros nomes. A convolução é a implementação de filtro linear
invariante à translação. Este filtro é totalmente especificado pelo núcleo (array $w$) que
contém os parâmetros do filtro linear (da rede convolucional) que precisam ser treinados.

A figura a seguir ilustra uma convolução da imagem verde de entrada, utilizando o núcleo da convolução (kernel) em amarelo, deslizando sobre a imagem verde. O resultado da convolução é a imagem rosa, de tamanho ligeiramente menor, devido ao processamento na borda da imagem.

![](http://mourafiq.com/images/posts/convolution_schematic.gif)

Esta próxima figura ilustra a convolução quando a imagem de saída (em verde) é igual às
dimensões da imagem de entrada (em azul). Neste caso, a imagem de entrada é artificialmente
aumentada com valores zeros. Este processo de aumentar a imagem de entrada com zeros é 
denominado *zero padding*.

![](https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/same_padding_no_strides.gif)

## Organização da imagem de entrada

As convoluções do Theano foram projetadas principalmente para tratar imagens ou sons. Assim, a organização dos arrays da imagem de entrada, de saída e dos pesos são feitos de forma específica para atender às necessidades das redes convolucionais. Iremos ilustrar as convoluções utilizando imagem de entrada em nível de cinza e coloridas com 3 bandas, usando
a convenção do *shape* de imagens do Theano. Diferentemente da organização utilizada nos classificadores usuais (não convolucionais), onde a organização da matriz $\boldsymbol{X}$ de entrada é bidimensional, com $n$ amostras nas linhas e $k$ *features* nas colunas. O shape de $\boldsymbol{X}$ é então

$$ (n,k)$$.
 
No caso de imagens, como elas têm
natureza bidimensional, a relação de vizinhança entre os pixels é fundamental, principalmente
para que os pesos dos núcleo abranjam uma região bidimensional da imagem. Assim, as *features* passam agora a terem duas dimensões: são $k$ pixels organizados em `(img_height, img_width)`. Desta forma o array passa a ter uma nova dimensão: ($n$, image_height, image_width). Numa 
imagem em nível de cinza temos apenas um valor para representar o pixel, que é sua intensidade, variando de escuro a claro. Quando a imagem é colorida, são necessários 3 valores
para cada pixel. No Theano, este 3 valores são organizados em uma dimensão adicional, denominada canal (*channel*) ou banda. A imagem colorida é organizada no *shape*:`(bandas, img_height, img_width)`. Desta forma o *shape* da imagem de entrada deve ter 4 dimensões:

$\boldsymbol{X}: $` (samples, channels, img_height, img_width)`

## Organização dos pesos (kernel) da convolução

Os pesos do kernel da convolução para tratar imagens devem ser 2D, quando a imagem
de entrada for em nível de cinza e devem ter 3 bandas 2D para imagens coloridas. Assim, a organização
do *shape* do kernel é $(bands, I_{height}, I_{width})$. Entretanto, nas arquiteturas das redes
convolucionais, é usual definir vários filtros (vários kernels), para que a rede tenha um
maior número de parâmetros a serem ajustados. Assim, o shape dos pesos devem ter 4 dimensões:

$\boldsymbol{W}:$ `(nb_filters, channels, img_height, img_width)`

Observe que a primeira dimensão (`nb_filters`) diz respeito ao número de filtros na saída,
enquanto que as três demais dimensões `(channels, img_height, img_width)` à janela deslizante
(*kernel*) da convolução.

## Organização da imagem de saída (filtrada)

A organização da imagem de saída é a mesma da imagem de entrada, até porque a camada convolucional pode ser colocada não apenas na entrada, mas como qualquer camada interna da
rede. A principal diferença está na nomenclatura da segunda dimensão: enquanto que na
imagem da camada de entrada a segunda dimensão é o *channel*, isto é o número de bandas 
da imagem colorida, na saída e demais camadas, a segunda dimensão passa a ser o número de
filtros aplicados naquela camada. Assim, o *shape* da imagem de saída fica como:

$\boldsymbol{Y}: $` (samples, filtros, img_height, img_width)`



## Analisando as convoluções em uma rede convolucional típica

Veja a rede a seguir com 4 camadas convolucionais:

![](../figures/architecture.png)
Fonte: [Figura apresentação do Sander Dieleman](http://benanne.github.io/images/architecture.png)

A imagem de entrada é uma imagem colorida
quadrada de 45 pixels de lado. Utiliza-se um kernel de 6x6 gerando 32 filtros, então os *shapes* são:
* Camada 1:
    - entrada: (amostras, 3, 45, 45)
    - kernel:  (32, 3, 6, 6)
    - saída: (amostras, 32, 40, 40) Obs: a imagem de saída fica menor
* Camada 2:
    - entrada: (amostras, 32, 40, 40) Obs: é a imagem de saída da camada anterior
    - kernel: (64, 32, 5, 5)
    - saída: (amostras, 64, 16, 16)
* Camada 3:
    - entrada: (amostras, 64, 16, 16)
    - kernel: (128, 64, 3, 3)
    - saída: (amostras, 128, 6, 6)
* Camada 4:
    - entrada: (amostras 128, 6, 6)
    - kernel: (128, 128, 3, 3)
    - saída: (amostras, 128, 4, 4)
    
## Cálculo do número de parâmetros no núcleos das convoluções

O número de parâmetros de cada convolução é igual ao número de elementos do kernel,
contabilizando todas as suas dimensões. Assim, um kernel de *shape* (10, 3, 5, 5) terá 750 elementos.

Como exercício, calcule o número de elementos dos kernels de cada uma das 4 camadas da rede
ilustrativa e o total de elementos.

Resposta:
- Camada 1:
- Camada 2:
- Camada 3:
- Camada 4:
- Total:

## Definição da convolução usando Keras

A implementação da convolução utiliza um modelo sequencial do Keras com `Convolution2D`.
Obtemos o resultado da rede pela função `predict`:

In [ ]:
# 4-D convolution
def conv4d(x,w,border_mode='same'):
    assert x.ndim == 4
    assert w.ndim == 4
    nb_filters,wd,wh,ww = w.shape
    samples,channels,xh,xw = x.shape
    model = Sequential([
                Convolution2D(nb_filters, wh, ww, 
                              input_shape=(channels, xh, xw),
                              weights=[W], 
                              border_mode=border_mode, bias=False, dim_ordering='th')
    ])
    y = model.predict(x,batch_size=1)
    return y


## Primeiro caso, resposta ao impulso

Quando a imagem de entrada é apenas um 1 rodeado de vários zeros, o resultado da convolução
é o próprio kernel posicionado no local do 1. Esta é uma forma usual para se verificar se
a convolução está funcionando. Assim, utilizaremos uma imagem cinza 3x5 com um "1" no centro.
O kernel é de 2 linhas e 3 colunas. O resultado é uma imagem 3x5 com o kernel posicionado
no local do 1 da imagem de entrada.

In [ ]:
np.set_printoptions(suppress=True, precision=3)

x = np.array([[0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0],
              [0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0]]).reshape(1,1,5,7)
W = np.array([[1.,2.,3.],
              [4.,5.,6.]]).reshape(1,1,2,3)

y = conv4d(x,W)

print 'x:', x.shape
print x
print 'y:', y.shape
print y
print 'kernel'
w = model.layers[0].get_weights()
print w



## Reproduzindo o exemplo da figura animada acima

In [ ]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
x = np.array([[1,1,1,0,0],
              [0,1,1,1,0],
              [0,0,1,1,1],
              [0,0,1,1,0],
              [0,1,1,0,0]]).reshape(1,1,5,5)
y = conv4d(x,W,'valid')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


## 4 amostras de imagem cinza 3x5, 1 filtro com kernel de 3x3 

Neste caso, 
- shape da entrada, x:(4,1,3,5)
- shape dos pesos W:(1, 1, 3,3). 
- shape de saída será y:(4,1,3,5)

In [ ]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
# criando 4 amostras de imagens
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5) * np.arange(4).reshape(4,1,1,1)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


## 2 filtros com kernel de 3x3, 1 amostra de imagem cinza 3x5

Neste caso, 
- shape da entrada, x:(1,1,3,5)
- shape dos pesos W:(2, 1, 3,3). 
- shape de saída será y:(1,2,3,5)


In [ ]:
W = np.array([[[1,0,1],
               [0,1,0],
               [1,0,1]],
              [[0,1,0],
               [1,2,1],
               [0,1,0]]]).reshape(2,1,3,3)
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


## 1 filtro com kernel de 3x3 e 3 canais, 1 amostra de imagem 3x5 RGB (3 canais)
Neste caso, como a imagem de entrada possui 3 bandas, para guardar as bandas Red, Green e Blue, o kernel também precisa ter 3 bandas pois são necessários pesos para cada banda de cor.
- shape da entrada, x:(1,3,3,5)
- shape dos pesos W:(1, 3, 3,3). 
- shape de saída será y:(1,2,3,5)


In [ ]:
W = np.array([[[1,0,1],
               [0,1,0],
               [1,0,1]],
              [[0,1,0],
               [1,1,1],
               [0,1,0]],
              [[0,1,0],
               [0,1,0],
               [0,1,0]]]).reshape(1,3,3,3)
x = np.array([[[0,0,0,0,0],
               [0,1,0,0,0],
               [0,0,0,0,0]],
              [[0,0,0,0,1],
               [0,0,1,0,0],
               [0,0,0,0,0]],
              [[0,0,0,0,0],
               [0,0,0,1,0],
               [0,0,0,0,0]]]).reshape(1,3,3,5)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'W:', W.shape
print W
print 'y:', y.shape
print y


## Exercícios

### Teórico

1. Supondo uma imagem de entrada 100 x 100 e um kernel 3 x 3. Utilizando-se o border_mode como válido, a imagem de saída será de 99 x 99. Se fosse utilizar uma rede densa, com entrada de 100 x 100 e saída 98 x 98, quantos parâmetros seriam necessários para ser treinados? E no caso da rede convolucional? Qual é o fator de redução?

### Práticos

1. Trocar o parâmetro ``border_mode`` para ``same`` e veja a diferença.
2. Rodar a convolução para uma imagem de cinza
3. Rodar a convolução para uma imagem colorida RGB
4. Comparar o tempo de execução desta convolução com outras implementações: scipy, skimage ou openCV